In [1]:
import numpy as np
import pandas as pd
import glob
import geopy
import geopy.distance
import datetime
import sys
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = r"/content/drive/My Drive/ship_data_files /train_ship_data_final_classifier.csv"
df=pd.read_csv(path,sep=',',low_memory=False)
df

Mounted at /content/drive


,Length,Breadth,Draught,Longitude,Latitude,SOG,bool,ETA
0,152,24,6.8,5.838777,53.63944,14.3,0,12.685507
1,152,24,6.8,5.845800,53.64076,14.3,0,12.667177
2,152,24,6.8,5.852149,53.64198,14.3,0,12.650496
3,152,24,6.8,5.858515,53.64320,14.3,0,12.633836
4,152,24,6.8,5.864876,53.64441,14.3,0,12.617179
...,...,...,...,...,...,...,...,...
186520,229,38,13.3,9.806900,53.55355,7.4,1,0.083519
186521,229,38,13.3,9.810224,53.55293,7.3,1,0.066774
186522,229,38,13.3,9.813442,53.55230,7.2,1,0.049974
186523,229,38,13.3,9.816650,53.55172,7.1,1,0.033443


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
df

,Length,Breadth,Draught,Longitude,Latitude,SOG,bool,ETA
0,152,24,6.8,5.838777,53.63944,14.3,0,12.685507
1,152,24,6.8,5.845800,53.64076,14.3,0,12.667177
2,152,24,6.8,5.852149,53.64198,14.3,0,12.650496
3,152,24,6.8,5.858515,53.64320,14.3,0,12.633836
4,152,24,6.8,5.864876,53.64441,14.3,0,12.617179
...,...,...,...,...,...,...,...,...
186520,229,38,13.3,9.806900,53.55355,7.4,1,0.083519
186521,229,38,13.3,9.810224,53.55293,7.3,1,0.066774
186522,229,38,13.3,9.813442,53.55230,7.2,1,0.049974
186523,229,38,13.3,9.816650,53.55172,7.1,1,0.033443


In [4]:
#Training part of the data
from sklearn import metrics

from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from hyperopt import fmin, tpe, hp,STATUS_OK, Trials
import matplotlib.pyplot as plt
!pip install tpot
from tpot import TPOTClassifier

     |████████████████████████████████| 92kB 2.6MB/s 
     |████████████████████████████████| 163kB 5.7MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=7ea220760de13fe04f9e4936c633009034c057ec0dcfbcfbb03a3c5ec3245009
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [5]:
#Features
features = df.drop('bool', axis=1)
features= np.array(features)

# Labels
labels= np.array(df['bool'])

feature_list = list(df.columns)

In [6]:
#Splitting the data to train and test

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [8]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:',  y_test.shape)



Training Features Shape: (139893, 7)
Training Labels Shape: (139893,)
Testing Features Shape: (46632, 7)
Testing Labels Shape: (46632,)


In [9]:
# Training part with logistic regression without tuning(Default values)

lr = LogisticRegression()

start_time = time.time()

lr.fit(X_train, y_train)

sv_notune= time.time() - start_time

y_pred= lr.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7864556527706296


In [10]:
# SVM object to be used to tune the default kernel will be 'rbf' 

lrc = LogisticRegression()



In [11]:
lrc.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [12]:
ITERATIONS=200
# Tuning LRC with the following algorithims
        # 1) Random
        # 2) Bayesian
        #  3) Genetic algorithims
        
# Random searcch set parameters
C=np.random.uniform(0.01,100,20)
max_iter = np.random.randint(10,ITERATIONS,20)
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty= ['none', 'l1', 'l2', 'elasticnet']
random_grid = {'C': C, 'solver':solver,
               'penalty': penalty,'max_iter':max_iter
               }
random_grid 

{'C': array([63.19906068, 15.95328385, 46.55616252, 65.0433648 , 29.11689704,
        36.56311207, 15.24417613, 52.0176739 , 16.26832282, 87.07309008,
        27.65807963, 73.33128475, 88.41024989, 96.15435029, 47.23173163,
        47.86617303, 16.67402855, 89.5259317 , 68.77297652, 39.60605373]),
 'max_iter': array([104,  93,  43,  53,  39, 135,  28,  66,  91,  61,  92, 113, 183,
        194,  78, 113, 105, 148,  81, 100]),
 'penalty': ['none', 'l1', 'l2', 'elasticnet'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [13]:
#Random search tuning
lr_random = RandomizedSearchCV(estimator = lrc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=42, n_jobs =-1)
start_time = time.time()
lr_random.fit(X_train, y_train)

random_time =time.time() - start_time

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  5

In [14]:
#Parameters selected for random search
lr_random.get_params()


{'cv': 3,
 'error_score': nan,
 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'estimator__C': 1.0,
 'estimator__class_weight': None,
 'estimator__dual': False,
 'estimator__fit_intercept': True,
 'estimator__intercept_scaling': 1,
 'estimator__l1_ratio': None,
 'estimator__max_iter': 100,
 'estimator__multi_class': 'auto',
 'estimator__n_jobs': None,
 'estimator__penalty': 'l2',
 'estimator__random_state': None,
 'estimator__solver': 'lbfgs',
 'estimator__tol': 0.0001,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'iid': 'deprecated',
 'n_iter': 100,
 'n_jobs': -1,
 'param_distributions': {'C': array([63.19906068, 15.95328385, 46.55616252, 65.0433648 , 29.11689704,
         36.5

In [15]:
#Testing on the test features, predictions for random search
y_pred1=lr_random.predict(X_test)

print("%s seconds it took to train with Random search tuning" % (random_time))
print(classification_report(y_test,y_pred1))

1036.9040744304657 seconds it took to train with Random search tuning
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     35954
           1       0.73      0.36      0.48     10678

    accuracy                           0.82     46632
   macro avg       0.78      0.66      0.69     46632
weighted avg       0.81      0.82      0.80     46632



In [16]:
#Tuning with Bayesian optimization

search_space= {'C': hp.uniform ('C', 0.01, 100),
        'max_iter': hp.uniform ('max_iter', 10, ITERATIONS)
    }
def objective(search_space):
    bayes = LogisticRegression(C = search_space['C'],max_iter = search_space['max_iter'])
    accuracy = cross_val_score(bayes, X_train, y_train, cv = 4).mean()
    return {'loss': accuracy, 'status': STATUS_OK }

trials = Trials()


# Training from bayes tuning


start_time = time.time()


best = fmin(fn= objective,
            space= search_space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)
best



trainedlrc  = LogisticRegression(C = best['C'],max_iter = best['max_iter']).fit(X_train, y_train)

bayesian_time =time.time() - start_time

y_pred2 = trainedlrc.predict(X_test)

  0%|          | 0/50 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

  2%|▏         | 1/50 [00:06<05:29,  6.73s/it, best loss: 0.8015697718161136]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

  4%|▍         | 2/50 [00:12<05:02,  6.31s/it, best loss: 0.7902682792977676]

...

 98%|█████████▊| 49/50 [03:32<00:05,  5.21s/it, best loss: 0.7739558091502005]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

100%|██████████| 50/50 [03:35<00:00,  4.31s/it, best loss: 0.7739558091502005]

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [17]:
# The genetic algorithim part

lrc_tpot = TPOTClassifier(verbosity=10, 
 scoring='accuracy', 
 random_state=20, 
 n_jobs=-1, 
 generations=5, 
 population_size=20,
 early_stop=5)

start_time = time.time()

lrc_tpot.fit(X_train, y_train) 

genetic_time = time.time() - start_time

32 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
Skipped pipeline #29 due to time out. Continuing to the next pipeline.
Skipped pipeline #31 due to time out. Continuing to the next pipeline.
Skipped pipeline #39 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 manhattan was provided as affinity. Ward can only work with euclidean distances..
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
Skipped pipeline #48 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=0 manhattan was provided as affinity. Ward can only work with euclid

In [18]:
predictions3 = lrc_tpot.predict(X_test)

In [19]:
# Results of training (time and classification report)

#1 Result of SVC without tuning
print("%s seconds it took to train without tuning" % (sv_notune))    
print(classification_report(y_test,y_pred))

#2 Result of SVC with Random search tuning
print("%s seconds it took to train with Random search tuning" % (random_time))
print(classification_report(y_test,y_pred1))

#3 Result of SVC with Bayesian search tuning 
print("%s seconds it took to train with Bayesian search tuning " % (bayesian_time))
print(classification_report(y_test,y_pred2))

#4 Result of Classification with Genetic Algorithim
print("%s seconds it took to train genetic algorithim" % (genetic_time))
print(classification_report(y_test,predictions3))


1.4375596046447754 seconds it took to train without tuning
              precision    recall  f1-score   support

           0       0.79      0.98      0.88     35954
           1       0.66      0.14      0.23     10678

    accuracy                           0.79     46632
   macro avg       0.73      0.56      0.55     46632
weighted avg       0.76      0.79      0.73     46632

1036.9040744304657 seconds it took to train with Random search tuning
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     35954
           1       0.73      0.36      0.48     10678

    accuracy                           0.82     46632
   macro avg       0.78      0.66      0.69     46632
weighted avg       0.81      0.82      0.80     46632

215.62035298347473 seconds it took to train with Bayesian search tuning 
              precision    recall  f1-score   support

           0       0.77      1.00      0.87     35954
           1       0.00      0.00  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
